## Data Acquirement

...

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nigeria2019election-analysis/summary-of-results.pdf


In [2]:
# Import necessary packages
import matplotlib.pyplot as plt #plotting and visulaizing data
import plotly.figure_factory as ff
import plotly.express as px
from bs4 import BeautifulSoup #scraping
import requests
import re
url="https://en.wikipedia.org/wiki/2019_Nigerian_general_election"
html_content = requests.get(url).text
# Parse HTML code for the entire site
soup = BeautifulSoup(html_content, "lxml")

/opt/conda/lib/python3.7/site-packages/geopandas/_compat.py:115: UserWarning: The Shapely GEOS version (3.9.1-CAPI-1.14.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


In [3]:
election_tables = soup.find_all("table", attrs={"class": "wikitable"})
print("Number of tables on site: ",len(election_tables))

Number of tables on site:  8


In [4]:
table1 = election_tables[4]
# the head will form our column names
body = table1.find_all("tr")
head = body[0] # 0th item is the header row
body_rows = body[1:] # All other items becomes the rest of the rows

headings = []
for item in head.find_all("th"): 
    # convert the th elements to text and strip "\n"
    item = (item.text).rstrip("\n")
    headings.append(item)
print(headings)

['State', 'Buhari', 'Atiku', 'Nicolas', 'Sowore', 'Moghalu', 'Durotoye', 'Duke', 'Mailafia']


In [5]:
all_rows = []
for row_num in range(len(body_rows)): 
    row = [] # this will old entries for one row
    for row_item in body_rows[row_num].find_all("td"): 
        # remove \xa0 and \n and comma from row_item.text
        aa = re.sub("(\xa0)|(\n)|,","",row_item.text)
        row.append(aa)
    # append one row to all_rows
    all_rows.append(row)

In [6]:
all_rows[:3]

[[],
 ['Abia',
  '85058',
  '26.31',
  '219698',
  '67.96',
  '1489',
  '0.46',
  '212',
  '',
  '88',
  '0.22',
  '720',
  '',
  '472',
  '',
  '336',
  ''],
 ['Adamawa',
  '378078',
  '46.59',
  '410266',
  '50.55',
  '3670',
  '0.45',
  '282',
  '',
  '109',
  '',
  '162',
  '',
  '978',
  '',
  '3989',
  '']]

In [7]:
new_rows = []
for row in all_rows:
    new_row = [t for i,t in enumerate(row) if i==0 or i%2!=0] #removing the empty space and the percentage
    new_rows.append(new_row)
print(new_rows)

[[], ['Abia', '85058', '219698', '1489', '212', '88', '720', '472', '336'], ['Adamawa', '378078', '410266', '3670', '282', '109', '162', '978', '3989'], ['Akwa Ibom', '175429', '395832', '1902', '222', '118', '95', '92', '230'], ['Anambra', '33298', '524738', '4374', '124', '4091', '45', '932', '227'], ['Bauchi', '798428 ', '209313', '2104', '183', '112', '46', '516', '296'], ['Bayelsa', '118821', '197933', '1584', '126', '50', '37', '124', '1078'], ['Benue', '347668', '356817', '2793', '309', '557', '201', '4927', '554'], ['Borno', '836496', '71788', '', '269', '78', '29', '322', '301'], ['Cross River', '117302', '295737', '', '242', '217', '88', '1395', '326'], ['Delta', '221292', '594068', '', '1626', '497', '320', '1745', '1075'], ['Ebonyi', '90726', '258573', '', '205', '192', '683', '452', '213'], ['Edo', '267842', '275691', '', '3106', '531', '273', '184', '850'], ['Ekiti', '219231', '154032', '', '400', '68', '88', '48', '406'], ['Enugu', '54423', '355553', '', '219', '1379', '

In [8]:
df = pd.DataFrame(data=new_rows,columns=headings)
df.head()

,State,Buhari,Atiku,Nicolas,Sowore,Moghalu,Durotoye,Duke,Mailafia
0,None,None,None,None,None,None,None,None,None
1,Abia,85058,219698,1489,212,88,720,472,336
2,Adamawa,378078,410266,3670,282,109,162,978,3989
3,Akwa Ibom,175429,395832,1902,222,118,95,92,230
4,Anambra,33298,524738,4374,124,4091,45,932,227


In [9]:
df.tail()

,State,Buhari,Atiku,Nicolas,Sowore,Moghalu,Durotoye,Duke,Mailafia
35,Yobe,497914,50763,,137,36,37,180,162
36,Zamfara,438682,125423,,186,44,24,81,186
37,FCT,152224,259997,,583,1083,652,410,246
38,Total,15191847,11262978,110196,33953,21886,16779,34746,97874
39,Source: BBC This Day Vanguard[49][50][51],None,None,None,None,None,None,None,None


In [10]:
df

,State,Buhari,Atiku,Nicolas,Sowore,Moghalu,Durotoye,Duke,Mailafia
0,None,None,None,None,None,None,None,None,None
1,Abia,85058,219698,1489,212,88,720,472,336
2,Adamawa,378078,410266,3670,282,109,162,978,3989
3,Akwa Ibom,175429,395832,1902,222,118,95,92,230
4,Anambra,33298,524738,4374,124,4091,45,932,227
5,Bauchi,798428,209313,2104,183,112,46,516,296
6,Bayelsa,118821,197933,1584,126,50,37,124,1078
7,Benue,347668,356817,2793,309,557,201,4927,554
8,Borno,836496,71788,,269,78,29,322,301
9,Cross River,117302,295737,,242,217,88,1395,326


**Nasarawa was wrongly spelled here as Nassarawa**

In [11]:
url="https://www.bbc.co.uk/news/resources/idt-f0b25208-4a1d-4068-a204-940cbe88d1d3"
html_content = requests.get(url).text
soup = BeautifulSoup(html_content, "lxml")
bbc_tables = soup.find_all("table", attrs={"class": "r-table__table"})
print("Number of tables on site: ",len(bbc_tables))

Number of tables on site:  1


In [12]:
df1=pd.read_html(str(bbc_tables))
# convert list to dataframe
df_=pd.DataFrame(df1[0])
print(df_.head())

       State     APC     PDP   PCP   ADC   APGA
0       Abia   85058  219698  1489   336   9638
1    Adamawa  378078  410266  3670  3989    159
2  Akwa Ibom  175429  395832  1902   230     61
3    Anambra   33298  524738  4374   227  30034
4     Bauchi  798428  209313  2104   296    149


In [13]:
df_

,State,APC,PDP,PCP,ADC,APGA
0,Abia,85058,219698,1489,336,9638
1,Adamawa,378078,410266,3670,3989,159
2,Akwa Ibom,175429,395832,1902,230,61
3,Anambra,33298,524738,4374,227,30034
4,Bauchi,798428,209313,2104,296,149
5,Bayelsa,118821,197933,1584,1078,53
6,Benue,347668,356817,2793,554,4582
7,Borno,836496,71788,1563,301,187
8,Cross River,117302,295737,2033,326,43
9,Delta,221292,594068,2753,1075,145


**Nasarawa appears here and spelled correctly**

In [14]:
election_data = pd.merge(df, df_, on='State', how='outer')

In [15]:
election_data.head()

,State,Buhari,Atiku,Nicolas,Sowore,Moghalu,Durotoye,Duke,Mailafia,APC,PDP,PCP,ADC,APGA
0,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN
1,Abia,85058,219698,1489,212,88,720,472,336,85058.0,219698.0,1489.0,336.0,9638.0
2,Adamawa,378078,410266,3670,282,109,162,978,3989,378078.0,410266.0,3670.0,3989.0,159.0
3,Akwa Ibom,175429,395832,1902,222,118,95,92,230,175429.0,395832.0,1902.0,230.0,61.0
4,Anambra,33298,524738,4374,124,4091,45,932,227,33298.0,524738.0,4374.0,227.0,30034.0


In [16]:
election_data.tail()

,State,Buhari,Atiku,Nicolas,Sowore,Moghalu,Durotoye,Duke,Mailafia,APC,PDP,PCP,ADC,APGA
37,FCT,152224,259997,,583,1083,652,410,246,152224.0,259997.0,2921.0,246.0,255.0
38,Total,15191847,11262978,110196,33953,21886,16779,34746,97874,NaN,NaN,NaN,NaN,NaN
39,Source: BBC This Day Vanguard[49][50][51],None,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN
40,Nasarawa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,289903.0,283847.0,1868.0,339.0,1523.0
41,Total:,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15191847.0,11262978.0,107286.0,97874.0,66851.0


In [17]:
election_data["State"].unique()

array([None, 'Abia', 'Adamawa', 'Akwa Ibom', 'Anambra', 'Bauchi',
       'Bayelsa', 'Benue', 'Borno', 'Cross River', 'Delta', 'Ebonyi',
       'Edo', 'Ekiti', 'Enugu', 'Gombe', 'Imo', 'Jigawa', 'Kaduna',
       'Kano', 'Katsina', 'Kebbi', 'Kogi', 'Kwara', 'Lagos', 'Nassarawa',
       'Niger', 'Ogun', 'Ondo', 'Osun', 'Oyo', 'Plateau', 'Rivers',
       'Sokoto', 'Taraba', 'Yobe', 'Zamfara', 'FCT', 'Total',
       'Source: BBC This Day Vanguard[49][50][51]', 'Nasarawa', 'Total:'],
      dtype=object)

In [18]:
election_data.shape

(42, 14)

In [19]:
election_data.to_csv("election_data.csv", index= False)

In [20]:
data = pd.read_csv("election_data.csv")

In [21]:
data

,State,Buhari,Atiku,Nicolas,Sowore,Moghalu,Durotoye,Duke,Mailafia,APC,PDP,PCP,ADC,APGA
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Abia,85058.0,219698.0,1489.0,212.0,88.0,720.0,472.0,336.0,85058.0,219698.0,1489.0,336.0,9638.0
2,Adamawa,378078.0,410266.0,3670.0,282.0,109.0,162.0,978.0,3989.0,378078.0,410266.0,3670.0,3989.0,159.0
3,Akwa Ibom,175429.0,395832.0,1902.0,222.0,118.0,95.0,92.0,230.0,175429.0,395832.0,1902.0,230.0,61.0
4,Anambra,33298.0,524738.0,4374.0,124.0,4091.0,45.0,932.0,227.0,33298.0,524738.0,4374.0,227.0,30034.0
5,Bauchi,798428.0,209313.0,2104.0,183.0,112.0,46.0,516.0,296.0,798428.0,209313.0,2104.0,296.0,149.0
6,Bayelsa,118821.0,197933.0,1584.0,126.0,50.0,37.0,124.0,1078.0,118821.0,197933.0,1584.0,1078.0,53.0
7,Benue,347668.0,356817.0,2793.0,309.0,557.0,201.0,4927.0,554.0,347668.0,356817.0,2793.0,554.0,4582.0
8,Borno,836496.0,71788.0,NaN,269.0,78.0,29.0,322.0,301.0,836496.0,71788.0,1563.0,301.0,187.0
9,Cross River,117302.0,295737.0,NaN,242.0,217.0,88.0,1395.0,326.0,117302.0,295737.0,2033.0,326.0,43.0


In [22]:
reg_female = [np.nan,961687, 897818, 1011636, 1232168, 1049945, 424392, 1174238, 971187, 754894, 1382716,
             799274, 1051209, 460789, 1047699, 589933, 1147356, 1016774, 1796282, 2422064,  
             1580464, 820397, 820687, 672396, 3013997, 771855, 999552, 1200561, 896454, 882545, 
              1478196, 1223204, 1490760, 806932, 806839, 552191, 787936, 601618, 39598645, np.nan, np.nan,np.nan
             ]

In [23]:
reg_male = [np.nan,971205, 1075265, 1108091, 1215828, 1412898, 498790, 1305893, 1344769, 772395, 1462558,
           660659, 1159325, 449178, 896317, 804460, 1124937, 1094332, 2136210, 3035683,
           1649766, 985834, 825663, 734061, 3556294, 845931, 1390483, 1174442, 925892, 797953,
            1455911, 1257251, 1724513, 1096234, 970266, 813722, 929192, 743238, 44405439, np.nan, np.nan, np.nan
           ]

In [24]:
data["reg_male"] = reg_male
data["reg_female"] = reg_female

In [25]:
data[["State", "reg_male", "reg_female"]]

,State,reg_male,reg_female
0,NaN,NaN,NaN
1,Abia,971205.0,961687.0
2,Adamawa,1075265.0,897818.0
3,Akwa Ibom,1108091.0,1011636.0
4,Anambra,1215828.0,1232168.0
5,Bauchi,1412898.0,1049945.0
6,Bayelsa,498790.0,424392.0
7,Benue,1305893.0,1174238.0
8,Borno,1344769.0,971187.0
9,Cross River,772395.0,754894.0


In [26]:
data["Total_reg_voters"] = data["reg_female"] + data["reg_male"]

In [27]:
data[["State", "Total_reg_voters"]]

,State,Total_reg_voters
0,NaN,NaN
1,Abia,1932892.0
2,Adamawa,1973083.0
3,Akwa Ibom,2119727.0
4,Anambra,2447996.0
5,Bauchi,2462843.0
6,Bayelsa,923182.0
7,Benue,2480131.0
8,Borno,2315956.0
9,Cross River,1527289.0


In [28]:
no_pvc_collectedn = [np.nan, 1729943, 1788706, 1933362, 2071714, 2335717, 769509, 2244376, 2000228, 1387314, 2470924,
                     1299048, 1726738, 666591, 1787537, 1335223, 1702178, 1625721, 3648831, 4696747,
                    3187988, 1718180, 1435751, 1149969, 5531389, 1442184, 2173204, 1694867, 1478460, 1266587,
                     2176352, 2095409, 2833101, 1726887, 1729094, 1261914, 1626839, 1026920,72775502, np.nan, np.nan, np.nan
]

In [29]:
data["No_pvc_collected"] = no_pvc_collectedn


In [30]:
data

,State,Buhari,Atiku,Nicolas,Sowore,Moghalu,Durotoye,Duke,Mailafia,APC,PDP,PCP,ADC,APGA,reg_male,reg_female,Total_reg_voters,No_pvc_collected
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Abia,85058.0,219698.0,1489.0,212.0,88.0,720.0,472.0,336.0,85058.0,219698.0,1489.0,336.0,9638.0,971205.0,961687.0,1932892.0,1729943.0
2,Adamawa,378078.0,410266.0,3670.0,282.0,109.0,162.0,978.0,3989.0,378078.0,410266.0,3670.0,3989.0,159.0,1075265.0,897818.0,1973083.0,1788706.0
3,Akwa Ibom,175429.0,395832.0,1902.0,222.0,118.0,95.0,92.0,230.0,175429.0,395832.0,1902.0,230.0,61.0,1108091.0,1011636.0,2119727.0,1933362.0
4,Anambra,33298.0,524738.0,4374.0,124.0,4091.0,45.0,932.0,227.0,33298.0,524738.0,4374.0,227.0,30034.0,1215828.0,1232168.0,2447996.0,2071714.0
5,Bauchi,798428.0,209313.0,2104.0,183.0,112.0,46.0,516.0,296.0,798428.0,209313.0,2104.0,296.0,149.0,1412898.0,1049945.0,2462843.0,2335717.0
6,Bayelsa,118821.0,197933.0,1584.0,126.0,50.0,37.0,124.0,1078.0,118821.0,197933.0,1584.0,1078.0,53.0,498790.0,424392.0,923182.0,769509.0
7,Benue,347668.0,356817.0,2793.0,309.0,557.0,201.0,4927.0,554.0,347668.0,356817.0,2793.0,554.0,4582.0,1305893.0,1174238.0,2480131.0,2244376.0
8,Borno,836496.0,71788.0,NaN,269.0,78.0,29.0,322.0,301.0,836496.0,71788.0,1563.0,301.0,187.0,1344769.0,971187.0,2315956.0,2000228.0
9,Cross River,117302.0,295737.0,NaN,242.0,217.0,88.0,1395.0,326.0,117302.0,295737.0,2033.0,326.0,43.0,772395.0,754894.0,1527289.0,1387314.0


In [31]:
data["No_pvc_uncollected"] = data['Total_reg_voters'] - data['No_pvc_collected']

In [32]:
data[['State', 'Total_reg_voters', 'No_pvc_uncollected']]

,State,Total_reg_voters,No_pvc_uncollected
0,NaN,NaN,NaN
1,Abia,1932892.0,202949.0
2,Adamawa,1973083.0,184377.0
3,Akwa Ibom,2119727.0,186365.0
4,Anambra,2447996.0,376282.0
5,Bauchi,2462843.0,127126.0
6,Bayelsa,923182.0,153673.0
7,Benue,2480131.0,235755.0
8,Borno,2315956.0,315728.0
9,Cross River,1527289.0,139975.0


In [33]:
age_distribution = {
    "Age Group": ["Youth", "Middle aged", 'Elderly', 'Old'],
    "Registered voters": [42938458, 25176144, 12788511, 3100971]  
}

- youth = 18-35
- middle-age = 36-50
- Elderly = 51-70
- old = 70+

In [34]:
df_age = pd.DataFrame(age_distribution)


In [35]:
df_age

,Age Group,Registered voters
0,Youth,42938458
1,Middle aged,25176144
2,Elderly,12788511
3,Old,3100971


In [36]:
occupation_distribution = {
    "Occupation": ["Artisan", "Business", 'Civil Servant', 'Farming/Fishing',
                  'House Wife', 'Other', 'Public Servant', 'Student', 'Trading'
                  ],
    "Registered voters": [4478202, 10810006, 5038671, 13630216, 
                          11844079, 6021741, 2292167, 22320990, 7568012
                         ]}

In [37]:
df_occupation = pd.DataFrame(occupation_distribution)

df_occupation

,Occupation,Registered voters
0,Artisan,4478202
1,Business,10810006
2,Civil Servant,5038671
3,Farming/Fishing,13630216
4,House Wife,11844079
5,Other,6021741
6,Public Servant,2292167
7,Student,22320990
8,Trading,7568012


## Data Cleaning

In [38]:
data

,State,Buhari,Atiku,Nicolas,Sowore,Moghalu,Durotoye,Duke,Mailafia,APC,PDP,PCP,ADC,APGA,reg_male,reg_female,Total_reg_voters,No_pvc_collected,No_pvc_uncollected
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Abia,85058.0,219698.0,1489.0,212.0,88.0,720.0,472.0,336.0,85058.0,219698.0,1489.0,336.0,9638.0,971205.0,961687.0,1932892.0,1729943.0,202949.0
2,Adamawa,378078.0,410266.0,3670.0,282.0,109.0,162.0,978.0,3989.0,378078.0,410266.0,3670.0,3989.0,159.0,1075265.0,897818.0,1973083.0,1788706.0,184377.0
3,Akwa Ibom,175429.0,395832.0,1902.0,222.0,118.0,95.0,92.0,230.0,175429.0,395832.0,1902.0,230.0,61.0,1108091.0,1011636.0,2119727.0,1933362.0,186365.0
4,Anambra,33298.0,524738.0,4374.0,124.0,4091.0,45.0,932.0,227.0,33298.0,524738.0,4374.0,227.0,30034.0,1215828.0,1232168.0,2447996.0,2071714.0,376282.0
5,Bauchi,798428.0,209313.0,2104.0,183.0,112.0,46.0,516.0,296.0,798428.0,209313.0,2104.0,296.0,149.0,1412898.0,1049945.0,2462843.0,2335717.0,127126.0
6,Bayelsa,118821.0,197933.0,1584.0,126.0,50.0,37.0,124.0,1078.0,118821.0,197933.0,1584.0,1078.0,53.0,498790.0,424392.0,923182.0,769509.0,153673.0
7,Benue,347668.0,356817.0,2793.0,309.0,557.0,201.0,4927.0,554.0,347668.0,356817.0,2793.0,554.0,4582.0,1305893.0,1174238.0,2480131.0,2244376.0,235755.0
8,Borno,836496.0,71788.0,NaN,269.0,78.0,29.0,322.0,301.0,836496.0,71788.0,1563.0,301.0,187.0,1344769.0,971187.0,2315956.0,2000228.0,315728.0
9,Cross River,117302.0,295737.0,NaN,242.0,217.0,88.0,1395.0,326.0,117302.0,295737.0,2033.0,326.0,43.0,772395.0,754894.0,1527289.0,1387314.0,139975.0


The first row isn't passing any message. Let's drop it.

In [39]:
data.drop([0], axis = 0, inplace= True)

In [40]:
data

,State,Buhari,Atiku,Nicolas,Sowore,Moghalu,Durotoye,Duke,Mailafia,APC,PDP,PCP,ADC,APGA,reg_male,reg_female,Total_reg_voters,No_pvc_collected,No_pvc_uncollected
1,Abia,85058.0,219698.0,1489.0,212.0,88.0,720.0,472.0,336.0,85058.0,219698.0,1489.0,336.0,9638.0,971205.0,961687.0,1932892.0,1729943.0,202949.0
2,Adamawa,378078.0,410266.0,3670.0,282.0,109.0,162.0,978.0,3989.0,378078.0,410266.0,3670.0,3989.0,159.0,1075265.0,897818.0,1973083.0,1788706.0,184377.0
3,Akwa Ibom,175429.0,395832.0,1902.0,222.0,118.0,95.0,92.0,230.0,175429.0,395832.0,1902.0,230.0,61.0,1108091.0,1011636.0,2119727.0,1933362.0,186365.0
4,Anambra,33298.0,524738.0,4374.0,124.0,4091.0,45.0,932.0,227.0,33298.0,524738.0,4374.0,227.0,30034.0,1215828.0,1232168.0,2447996.0,2071714.0,376282.0
5,Bauchi,798428.0,209313.0,2104.0,183.0,112.0,46.0,516.0,296.0,798428.0,209313.0,2104.0,296.0,149.0,1412898.0,1049945.0,2462843.0,2335717.0,127126.0
6,Bayelsa,118821.0,197933.0,1584.0,126.0,50.0,37.0,124.0,1078.0,118821.0,197933.0,1584.0,1078.0,53.0,498790.0,424392.0,923182.0,769509.0,153673.0
7,Benue,347668.0,356817.0,2793.0,309.0,557.0,201.0,4927.0,554.0,347668.0,356817.0,2793.0,554.0,4582.0,1305893.0,1174238.0,2480131.0,2244376.0,235755.0
8,Borno,836496.0,71788.0,NaN,269.0,78.0,29.0,322.0,301.0,836496.0,71788.0,1563.0,301.0,187.0,1344769.0,971187.0,2315956.0,2000228.0,315728.0
9,Cross River,117302.0,295737.0,NaN,242.0,217.0,88.0,1395.0,326.0,117302.0,295737.0,2033.0,326.0,43.0,772395.0,754894.0,1527289.0,1387314.0,139975.0
10,Delta,221292.0,594068.0,NaN,1626.0,497.0,320.0,1745.0,1075.0,221292.0,594068.0,2753.0,1075.0,145.0,1462558.0,1382716.0,2845274.0,2470924.0,374350.0


- row 39 is the source of that(as stated by Wikipedia). This information is not needed here, so we drop it. 
- 38 and 40 are the sums of the columns, this can be easily calculated when needed, so the two rows should be dropped.
- Let's drop row 40 since we have another row(Nassarawa) with the same data

In [41]:
data.drop([38,39,40,41], axis = 0, inplace = True)

In [42]:
data

,State,Buhari,Atiku,Nicolas,Sowore,Moghalu,Durotoye,Duke,Mailafia,APC,PDP,PCP,ADC,APGA,reg_male,reg_female,Total_reg_voters,No_pvc_collected,No_pvc_uncollected
1,Abia,85058.0,219698.0,1489.0,212.0,88.0,720.0,472.0,336.0,85058.0,219698.0,1489.0,336.0,9638.0,971205.0,961687.0,1932892.0,1729943.0,202949.0
2,Adamawa,378078.0,410266.0,3670.0,282.0,109.0,162.0,978.0,3989.0,378078.0,410266.0,3670.0,3989.0,159.0,1075265.0,897818.0,1973083.0,1788706.0,184377.0
3,Akwa Ibom,175429.0,395832.0,1902.0,222.0,118.0,95.0,92.0,230.0,175429.0,395832.0,1902.0,230.0,61.0,1108091.0,1011636.0,2119727.0,1933362.0,186365.0
4,Anambra,33298.0,524738.0,4374.0,124.0,4091.0,45.0,932.0,227.0,33298.0,524738.0,4374.0,227.0,30034.0,1215828.0,1232168.0,2447996.0,2071714.0,376282.0
5,Bauchi,798428.0,209313.0,2104.0,183.0,112.0,46.0,516.0,296.0,798428.0,209313.0,2104.0,296.0,149.0,1412898.0,1049945.0,2462843.0,2335717.0,127126.0
6,Bayelsa,118821.0,197933.0,1584.0,126.0,50.0,37.0,124.0,1078.0,118821.0,197933.0,1584.0,1078.0,53.0,498790.0,424392.0,923182.0,769509.0,153673.0
7,Benue,347668.0,356817.0,2793.0,309.0,557.0,201.0,4927.0,554.0,347668.0,356817.0,2793.0,554.0,4582.0,1305893.0,1174238.0,2480131.0,2244376.0,235755.0
8,Borno,836496.0,71788.0,NaN,269.0,78.0,29.0,322.0,301.0,836496.0,71788.0,1563.0,301.0,187.0,1344769.0,971187.0,2315956.0,2000228.0,315728.0
9,Cross River,117302.0,295737.0,NaN,242.0,217.0,88.0,1395.0,326.0,117302.0,295737.0,2033.0,326.0,43.0,772395.0,754894.0,1527289.0,1387314.0,139975.0
10,Delta,221292.0,594068.0,NaN,1626.0,497.0,320.0,1745.0,1075.0,221292.0,594068.0,2753.0,1075.0,145.0,1462558.0,1382716.0,2845274.0,2470924.0,374350.0


Cool, we have 37 states in Nigeria

> Correcting the spelling of `Nasarawa` spelled as `'Nassarawa'`

In [43]:
data.replace(to_replace= "Nassarawa", value='Nasarawa', inplace= True)

In [44]:
data.head()

,State,Buhari,Atiku,Nicolas,Sowore,Moghalu,Durotoye,Duke,Mailafia,APC,PDP,PCP,ADC,APGA,reg_male,reg_female,Total_reg_voters,No_pvc_collected,No_pvc_uncollected
1,Abia,85058.0,219698.0,1489.0,212.0,88.0,720.0,472.0,336.0,85058.0,219698.0,1489.0,336.0,9638.0,971205.0,961687.0,1932892.0,1729943.0,202949.0
2,Adamawa,378078.0,410266.0,3670.0,282.0,109.0,162.0,978.0,3989.0,378078.0,410266.0,3670.0,3989.0,159.0,1075265.0,897818.0,1973083.0,1788706.0,184377.0
3,Akwa Ibom,175429.0,395832.0,1902.0,222.0,118.0,95.0,92.0,230.0,175429.0,395832.0,1902.0,230.0,61.0,1108091.0,1011636.0,2119727.0,1933362.0,186365.0
4,Anambra,33298.0,524738.0,4374.0,124.0,4091.0,45.0,932.0,227.0,33298.0,524738.0,4374.0,227.0,30034.0,1215828.0,1232168.0,2447996.0,2071714.0,376282.0
5,Bauchi,798428.0,209313.0,2104.0,183.0,112.0,46.0,516.0,296.0,798428.0,209313.0,2104.0,296.0,149.0,1412898.0,1049945.0,2462843.0,2335717.0,127126.0


Some presidential candidates and their party:


|Candidate | Party | Occured twice in the table |
| :-------- | :-----: | -----: |
|Muhammadu Buhari|  APC | True |
|Atiku Abubakar| PDP |  True |
|Felix Nicolas| PCP | True |
|Obadiah Mailafia| ADC | True |
|Donald Duke| SDP | False |
|John Gbor| APGA | False |
|Omoyele Sowore| AAC | False |
|Fela Durotoye| ANN | False |
|Kingsley Moghalu| YPP | False |

- `Buhari`(APC), `Atiku`(PDP), `Nicholas`(PCP), and `Mailafia`(ADC) occured twice as 2 columns in the data. Let's the candidate's name and leave the party since they contains the same values.

In [45]:
data.drop(['Buhari', 'Atiku', 'Nicolas', 'Mailafia'], axis = 1, inplace=True)

In [46]:
data

,State,Sowore,Moghalu,Durotoye,Duke,APC,PDP,PCP,ADC,APGA,reg_male,reg_female,Total_reg_voters,No_pvc_collected,No_pvc_uncollected
1,Abia,212.0,88.0,720.0,472.0,85058.0,219698.0,1489.0,336.0,9638.0,971205.0,961687.0,1932892.0,1729943.0,202949.0
2,Adamawa,282.0,109.0,162.0,978.0,378078.0,410266.0,3670.0,3989.0,159.0,1075265.0,897818.0,1973083.0,1788706.0,184377.0
3,Akwa Ibom,222.0,118.0,95.0,92.0,175429.0,395832.0,1902.0,230.0,61.0,1108091.0,1011636.0,2119727.0,1933362.0,186365.0
4,Anambra,124.0,4091.0,45.0,932.0,33298.0,524738.0,4374.0,227.0,30034.0,1215828.0,1232168.0,2447996.0,2071714.0,376282.0
5,Bauchi,183.0,112.0,46.0,516.0,798428.0,209313.0,2104.0,296.0,149.0,1412898.0,1049945.0,2462843.0,2335717.0,127126.0
6,Bayelsa,126.0,50.0,37.0,124.0,118821.0,197933.0,1584.0,1078.0,53.0,498790.0,424392.0,923182.0,769509.0,153673.0
7,Benue,309.0,557.0,201.0,4927.0,347668.0,356817.0,2793.0,554.0,4582.0,1305893.0,1174238.0,2480131.0,2244376.0,235755.0
8,Borno,269.0,78.0,29.0,322.0,836496.0,71788.0,1563.0,301.0,187.0,1344769.0,971187.0,2315956.0,2000228.0,315728.0
9,Cross River,242.0,217.0,88.0,1395.0,117302.0,295737.0,2033.0,326.0,43.0,772395.0,754894.0,1527289.0,1387314.0,139975.0
10,Delta,1626.0,497.0,320.0,1745.0,221292.0,594068.0,2753.0,1075.0,145.0,1462558.0,1382716.0,2845274.0,2470924.0,374350.0


> Let's replace the candidate name with their party's name.

|Candidate | Party | 
| :-------- | :-----: |
|Donald Duke| SDP |  
|John Gbor| APGA | 
|Omoyele Sowore| AAC |
|Fela Durotoye| ANN |
|Kingsley Moghalu| YPP |

> `John Gbor` of APGA is not on the table, but APGA is. There's no column to rename for ~Gbor~

In [47]:
data.rename({"Sowore": "AAC", "Moghalu": "YPP", "Durotoye": "ANN", "Duke": "SDP" }, axis = 1, inplace = True)

In [48]:
data.head()

,State,AAC,YPP,ANN,SDP,APC,PDP,PCP,ADC,APGA,reg_male,reg_female,Total_reg_voters,No_pvc_collected,No_pvc_uncollected
1,Abia,212.0,88.0,720.0,472.0,85058.0,219698.0,1489.0,336.0,9638.0,971205.0,961687.0,1932892.0,1729943.0,202949.0
2,Adamawa,282.0,109.0,162.0,978.0,378078.0,410266.0,3670.0,3989.0,159.0,1075265.0,897818.0,1973083.0,1788706.0,184377.0
3,Akwa Ibom,222.0,118.0,95.0,92.0,175429.0,395832.0,1902.0,230.0,61.0,1108091.0,1011636.0,2119727.0,1933362.0,186365.0
4,Anambra,124.0,4091.0,45.0,932.0,33298.0,524738.0,4374.0,227.0,30034.0,1215828.0,1232168.0,2447996.0,2071714.0,376282.0
5,Bauchi,183.0,112.0,46.0,516.0,798428.0,209313.0,2104.0,296.0,149.0,1412898.0,1049945.0,2462843.0,2335717.0,127126.0


let's chheck for `NaN` values

In [49]:
data[data.isnull().any(axis=1)]

,State,AAC,YPP,ANN,SDP,APC,PDP,PCP,ADC,APGA,reg_male,reg_female,Total_reg_voters,No_pvc_collected,No_pvc_uncollected
25,Nasarawa,75.0,44.0,45.0,359.0,NaN,NaN,NaN,NaN,NaN,845931.0,771855.0,1617786.0,1442184.0,175602.0


Oops!

We shouldn't have NaN value in an election result, beacuse at least one person would vote for a party in a state.
How can the 2 major political parties have no vote in `Nasarawa`?
> This error occured because of the row 38 and column `Buhari`, `Atiku` and co that we dropped.

Let's correct the error.


In [50]:
#df_ is the BBC dataframe above
df_

,State,APC,PDP,PCP,ADC,APGA
0,Abia,85058,219698,1489,336,9638
1,Adamawa,378078,410266,3670,3989,159
2,Akwa Ibom,175429,395832,1902,230,61
3,Anambra,33298,524738,4374,227,30034
4,Bauchi,798428,209313,2104,296,149
5,Bayelsa,118821,197933,1584,1078,53
6,Benue,347668,356817,2793,554,4582
7,Borno,836496,71788,1563,301,187
8,Cross River,117302,295737,2033,326,43
9,Delta,221292,594068,2753,1075,145


**Let's fill the NaN values with its corresponding value in the df_ table**

In [51]:
#Filling the NaN values with its corresponding value in the df_ table
data['APC'].fillna(df_["APC"], inplace=True)
data['PDP'].fillna(df_["PDP"], inplace=True)
data['PCP'].fillna(df_["PCP"], inplace=True)
data['ADC'].fillna(df_["ADC"], inplace=True)
data['APGA'].fillna(df_["APGA"], inplace=True)


In [52]:
data

,State,AAC,YPP,ANN,SDP,APC,PDP,PCP,ADC,APGA,reg_male,reg_female,Total_reg_voters,No_pvc_collected,No_pvc_uncollected
1,Abia,212.0,88.0,720.0,472.0,85058.0,219698.0,1489.0,336.0,9638.0,971205.0,961687.0,1932892.0,1729943.0,202949.0
2,Adamawa,282.0,109.0,162.0,978.0,378078.0,410266.0,3670.0,3989.0,159.0,1075265.0,897818.0,1973083.0,1788706.0,184377.0
3,Akwa Ibom,222.0,118.0,95.0,92.0,175429.0,395832.0,1902.0,230.0,61.0,1108091.0,1011636.0,2119727.0,1933362.0,186365.0
4,Anambra,124.0,4091.0,45.0,932.0,33298.0,524738.0,4374.0,227.0,30034.0,1215828.0,1232168.0,2447996.0,2071714.0,376282.0
5,Bauchi,183.0,112.0,46.0,516.0,798428.0,209313.0,2104.0,296.0,149.0,1412898.0,1049945.0,2462843.0,2335717.0,127126.0
6,Bayelsa,126.0,50.0,37.0,124.0,118821.0,197933.0,1584.0,1078.0,53.0,498790.0,424392.0,923182.0,769509.0,153673.0
7,Benue,309.0,557.0,201.0,4927.0,347668.0,356817.0,2793.0,554.0,4582.0,1305893.0,1174238.0,2480131.0,2244376.0,235755.0
8,Borno,269.0,78.0,29.0,322.0,836496.0,71788.0,1563.0,301.0,187.0,1344769.0,971187.0,2315956.0,2000228.0,315728.0
9,Cross River,242.0,217.0,88.0,1395.0,117302.0,295737.0,2033.0,326.0,43.0,772395.0,754894.0,1527289.0,1387314.0,139975.0
10,Delta,1626.0,497.0,320.0,1745.0,221292.0,594068.0,2753.0,1075.0,145.0,1462558.0,1382716.0,2845274.0,2470924.0,374350.0


In [53]:
votes = [{'State': 'Osun',
'Total No Accredited': 732984,
'Total Votes Cast': 731882,
'Total Valid Votes': 714682,
'Rejected Votes': 17200},
 
{
    'State': 'Ekiti' ,
'Total No Accredited': 395741,
'Total Votes Cast': 393709,
'Total Valid Votes': 381132,
'Rejected Votes': 12577
},
 
{
'State': 'FCT',
'Total No Accredited': 467784,
'Total Votes Cast': 451408,
'Total Valid Votes': 423951,
'Rejected Votes': 27457
},
 
{
'State':     'Kwara', 
'Total No Accredited': 489482,
'Total Votes Cast': 486254,
'Total Valid Votes': 459676,
'Rejected Votes': 26578
},

 {
'State':     'Nasarawa',
'Total No Accredited': 613720,
'Total Votes Cast': 599399,
'Total Valid Votes': 580778,
'Rejected Votes': 18621
 },
 
 {
'State':      'Kogi',
'Total No Accredited': 570773,
'Total Votes Cast': 553496,
'Total Valid Votes': 521016,
'Rejected Votes': 32480
 },
 
{ 'State':  'Gombe',
'Total No Accredited': 604240,
'Total Votes Cast': 580649,
'Total Valid Votes': 554203,
'Rejected Votes': 26446
},
 
 {
'State':      'Ondo', 
'Total No Accredited': 598586,
'Total Votes Cast': 586827,
'Total Valid Votes': 555994,
'Rejected Votes': 30833 
 },
 
 {
'State':     'Abia', 
'Total No Accredited': 361561,
'Total Votes Cast': 344471,
'Total Valid Votes': 323291,
'Rejected Votes': 21180
 },
 
 {
'State':    'Yobe',
'Total No Accredited': 601059,
'Total Votes Cast': 586137,
'Total Valid Votes': 559365,
'Rejected Votes': 26772  
 },
 {
'State':  'Enugu', 
'Total No Accredited': 452765,
'Total Votes Cast': 451063,
'Total Valid Votes': 421014,
'Rejected Votes': 30049  
 },
{
'State': 'Ebonyi',
'Total No Accredited': 391747,
'Total Votes Cast': 379394,
'Total Valid Votes': 359131,
'Rejected Votes': 20263
},
{
'State': 'Niger',
'Total No Accredited': 911964,
'Total Votes Cast': 896976,
'Total Valid Votes': 851937,
'Rejected Votes':45039
},
{
'State': 'Jigawa',
'Total No Accredited': 1171801,
'Total Votes Cast': 1149922,
'Total Valid Votes': 1106244,
'Rejected Votes': 43678
},
{
'State':    'Kaduna',
'Total No Accredited': 1757868,
'Total Votes Cast': 1709005,
'Total Valid Votes': 1663603,
'Rejected Votes': 45402 
},
{
'State': 'Anambra', 
'Total No Accredited': 675273,
'Total Votes Cast': 625035,
'Total Valid Votes':605734,
'Rejected Votes':19301
},
{
'State': 'Oyo',
'Total No Accredited': 905007,
'Total Votes Cast': 891080,
'Total Valid Votes': 836531,
'Rejected Votes': 54549
},
 
{
'State': 'Adamawa',
'Total No Accredited':874920,
'Total Votes Cast':860756,
'Total Valid Votes':811534,
'Rejected Votes':49222
},
 {
'State': 'Ogun',
'Total No Accredited': 613397,
'Total Votes Cast': 605938,
'Total Valid Votes': 564256,
'Rejected Votes': 41682
 },
 {
'State': 'Lagos',
'Total No Accredited': 1196490,
'Total Votes Cast': 1156590,
'Total Valid Votes': 1089567,
'Rejected Votes': 67023
 },
 {
'State': 'Bauchi',
'Total No Accredited':1075330,
'Total Votes Cast':1061955,
'Total Valid Votes':1024307,
'Rejected Votes':37648
 },
 {
'State': 'Edo',
'Total No Accredited': 604915,
'Total Votes Cast': 599228,
'Total Valid Votes': 560711,
'Rejected Votes': 38517
 },

 {
'State': 'Benue',
'Total No Accredited': 786069,
'Total Votes Cast': 763872,
'Total Valid Votes': 728912,
'Rejected Votes': 34960
 },
 {
'State': 'Imo',
'Total No Accredited': 585741,
'Total Votes Cast': 542777,
'Total Valid Votes': 511586,
'Rejected Votes': 31191
 },
 {
'State': 'Plateau',
'Total No Accredited':1074042,
'Total Votes Cast':1062862,
'Total Valid Votes':1034853,
'Rejected Votes':28009
 },
 {
'State': 'Kano',
'Total No Accredited':2006410,
'Total Votes Cast':1964751,
'Total Valid Votes':1891134,
'Rejected Votes':73617
 },
 {
'State': 'Katsina',
'Total No Accredited':1628865,
'Total Votes Cast':1619185,
'Total Valid Votes':1555473,
'Rejected Votes':63712
 },
 {
'State': 'Taraba', 
'Total No Accredited': 756111,
'Total Votes Cast': 741564,
'Total Valid Votes': 712877,
'Rejected Votes': 28687
 },
 {
'State': 'Cross River',
'Total No Accredited': 461033,
'Total Votes Cast': 446046,
'Total Valid Votes': 421901,
'Rejected Votes': 24145
 },
 {
'State': 'Akwa Ibom',
'Total No Accredited': 695677,
'Total Votes Cast': 605140,
'Total Valid Votes': 578775,
'Rejected Votes': 26365
 },
 {
'State': 'Delta',
'Total No Accredited': 891647,
'Total Votes Cast': 882254,
'Total Valid Votes': 829762,
'Rejected Votes': 52492
 },
 {
'State': 'Borno',
'Total No Accredited': 987290,
'Total Votes Cast': 955205,
'Total Valid Votes': 919786,
'Rejected Votes': 35419
 },
{'State':  'Rivers', 
'Total No Accredited':678167,
'Total Votes Cast':666585,
'Total Valid Votes':642165,
'Rejected Votes': 24420},
 
 {
'State': 'Zamfara',
'Total No Accredited': 616168,
'Total Votes Cast': 597224,
'Total Valid Votes': 578439,
'Rejected Votes': 18785
 },
 {
'State': 'Kebbi',
'Total No Accredited': 835238,
'Total Votes Cast':803755,
'Total Valid Votes':756605,
'Rejected Votes':47150
 },
{
'State': 'Bayelsa',
'Total No Accredited':344237,
'Total Votes Cast':335856,
'Total Valid Votes':321767,
'Rejected Votes':14089
},

{
'State': 'Sokoto',
'Total No Accredited': 950107,
'Total Votes Cast': 925940,
'Total Valid Votes': 871891,
'Rejected Votes': 54049
}
]

In [54]:
votes_df = pd.DataFrame(votes) 

In [55]:
votes_df

,State,Total No Accredited,Total Votes Cast,Total Valid Votes,Rejected Votes
0,Osun,732984,731882,714682,17200
1,Ekiti,395741,393709,381132,12577
2,FCT,467784,451408,423951,27457
3,Kwara,489482,486254,459676,26578
4,Nasarawa,613720,599399,580778,18621
5,Kogi,570773,553496,521016,32480
6,Gombe,604240,580649,554203,26446
7,Ondo,598586,586827,555994,30833
8,Abia,361561,344471,323291,21180
9,Yobe,601059,586137,559365,26772


In [56]:
votes_df=votes_df.sort_values(by='State', axis= 0)

In [57]:
votes_df

,State,Total No Accredited,Total Votes Cast,Total Valid Votes,Rejected Votes
8,Abia,361561,344471,323291,21180
17,Adamawa,874920,860756,811534,49222
29,Akwa Ibom,695677,605140,578775,26365
15,Anambra,675273,625035,605734,19301
20,Bauchi,1075330,1061955,1024307,37648
35,Bayelsa,344237,335856,321767,14089
22,Benue,786069,763872,728912,34960
31,Borno,987290,955205,919786,35419
28,Cross River,461033,446046,421901,24145
30,Delta,891647,882254,829762,52492


In [58]:
final_data = pd.merge(data, votes_df, on='State', how='outer')

In [59]:
final_data.head()

,State,AAC,YPP,ANN,SDP,APC,PDP,PCP,ADC,APGA,reg_male,reg_female,Total_reg_voters,No_pvc_collected,No_pvc_uncollected,Total No Accredited,Total Votes Cast,Total Valid Votes,Rejected Votes
0,Abia,212.0,88.0,720.0,472.0,85058.0,219698.0,1489.0,336.0,9638.0,971205.0,961687.0,1932892.0,1729943.0,202949.0,361561,344471,323291,21180
1,Adamawa,282.0,109.0,162.0,978.0,378078.0,410266.0,3670.0,3989.0,159.0,1075265.0,897818.0,1973083.0,1788706.0,184377.0,874920,860756,811534,49222
2,Akwa Ibom,222.0,118.0,95.0,92.0,175429.0,395832.0,1902.0,230.0,61.0,1108091.0,1011636.0,2119727.0,1933362.0,186365.0,695677,605140,578775,26365
3,Anambra,124.0,4091.0,45.0,932.0,33298.0,524738.0,4374.0,227.0,30034.0,1215828.0,1232168.0,2447996.0,2071714.0,376282.0,675273,625035,605734,19301
4,Bauchi,183.0,112.0,46.0,516.0,798428.0,209313.0,2104.0,296.0,149.0,1412898.0,1049945.0,2462843.0,2335717.0,127126.0,1075330,1061955,1024307,37648


In [60]:
!pip install geopy 


In [61]:
from geopy.exc import GeocoderTimedOut
from geopy.geocoders import Nominatim
   

def findGeocode(state):
       
    # try and catch is used to overcome
    # the exception thrown by geolocator
    # using geocodertimedout  
    try:
          
        # Specify the user_agent as your
        # app name it should not be none
        geolocator = Nominatim(user_agent="your_app_name")
          
        return geolocator.geocode(state)
      
    except GeocoderTimedOut:
          
        return findGeocode(state)    

In [62]:
# declare an empty list to store
# latitude and longitude of values 
# of city column
longitude = []
latitude = []
# each value from city column
# will be fetched and sent to
# function find_geocode   
for i in (final_data["State"]):
      
    if findGeocode(i) != None:
           
        loc = findGeocode(i)
          
        # coordinates returned from 
        # function is stored into
        # two separate list
        latitude.append(loc.latitude)
        longitude.append(loc.longitude)
       
    # if coordinate for a city not
    # found, insert "NaN" indicating 
    # missing value 
    else:
        latitude.append(np.nan)
        longitude.append(np.nan)

In [63]:
final_data["Latitude"] = latitude
final_data["Longitude"] = longitude

In [64]:
final_data

,State,AAC,YPP,ANN,SDP,APC,PDP,PCP,ADC,APGA,...,reg_female,Total_reg_voters,No_pvc_collected,No_pvc_uncollected,Total No Accredited,Total Votes Cast,Total Valid Votes,Rejected Votes,Latitude,Longitude
0,Abia,212.0,88.0,720.0,472.0,85058.0,219698.0,1489.0,336.0,9638.0,...,961687.0,1932892.0,1729943.0,202949.0,361561,344471,323291,21180,5.454095,7.515307
1,Adamawa,282.0,109.0,162.0,978.0,378078.0,410266.0,3670.0,3989.0,159.0,...,897818.0,1973083.0,1788706.0,184377.0,874920,860756,811534,49222,9.512977,12.388189
2,Akwa Ibom,222.0,118.0,95.0,92.0,175429.0,395832.0,1902.0,230.0,61.0,...,1011636.0,2119727.0,1933362.0,186365.0,695677,605140,578775,26365,4.988061,7.795178
3,Anambra,124.0,4091.0,45.0,932.0,33298.0,524738.0,4374.0,227.0,30034.0,...,1232168.0,2447996.0,2071714.0,376282.0,675273,625035,605734,19301,6.218314,6.953184
4,Bauchi,183.0,112.0,46.0,516.0,798428.0,209313.0,2104.0,296.0,149.0,...,1049945.0,2462843.0,2335717.0,127126.0,1075330,1061955,1024307,37648,10.622828,10.028775
5,Bayelsa,126.0,50.0,37.0,124.0,118821.0,197933.0,1584.0,1078.0,53.0,...,424392.0,923182.0,769509.0,153673.0,344237,335856,321767,14089,4.762979,6.028898
6,Benue,309.0,557.0,201.0,4927.0,347668.0,356817.0,2793.0,554.0,4582.0,...,1174238.0,2480131.0,2244376.0,235755.0,786069,763872,728912,34960,7.350575,8.777288
7,Borno,269.0,78.0,29.0,322.0,836496.0,71788.0,1563.0,301.0,187.0,...,971187.0,2315956.0,2000228.0,315728.0,987290,955205,919786,35419,45.956665,10.174282
8,Cross River,242.0,217.0,88.0,1395.0,117302.0,295737.0,2033.0,326.0,43.0,...,754894.0,1527289.0,1387314.0,139975.0,461033,446046,421901,24145,5.867197,8.520477
9,Delta,1626.0,497.0,320.0,1745.0,221292.0,594068.0,2753.0,1075.0,145.0,...,1382716.0,2845274.0,2470924.0,374350.0,891647,882254,829762,52492,33.392689,-95.674949
